In [1]:
import chess

In [2]:
# defining the black and white chess pieces on the chess board
Black = {
    chess.PAWN: '♟',
    chess.KNIGHT: '♞',
    chess.BISHOP: '♝',
    chess.ROOK: '♜',
    chess.QUEEN: '♛',
    chess.KING: '♚'
}

White = {
    chess.PAWN: '♙',
    chess.KNIGHT: '♘',
    chess.BISHOP: '♗',
    chess.ROOK: '♖',
    chess.QUEEN: '♕',
    chess.KING: '♔'
}
chessboard = chess.Board()


In [3]:
def analyzeState(board):
    points = 0

    # Assigning values to the pieces
    for piece, value in [(chess.PAWN, 1), (chess.KNIGHT, 3), (chess.BISHOP, 3),
                         (chess.ROOK, 5), (chess.QUEEN, 9)]:
        points += value * (len(board.pieces(piece, chess.WHITE)) - len(board.pieces(piece, chess.BLACK)))


    # Checking the killing moves and giving them extra points
    for move in board.legal_moves:
        if board.is_capture(move):
            captured_square = move.to_square
            captured_piece = board.piece_at(captured_square)
            if captured_piece is not None:
                if captured_piece.piece_type == chess.PAWN:
                    points += 1
                elif captured_piece.piece_type == chess.KNIGHT:
                    points += 3
                elif captured_piece.piece_type == chess.BISHOP:
                    points += 3
                elif captured_piece.piece_type == chess.ROOK:
                    points += 5
                elif captured_piece.piece_type == chess.QUEEN:
                    points += 9

    # Define a dictionary of initial squares for each piece type
    INITIAL_SQUARES = {chess.PAWN: [sq for sq in chess.SQUARES_180 if 48 <= sq <= 55],
                       chess.KNIGHT: [chess.G1, chess.B1, chess.G8, chess.B8],
                       chess.BISHOP: [chess.F1, chess.C1, chess.F8, chess.C8],
                       chess.ROOK: [chess.H1, chess.A1, chess.H8, chess.A8],
                       chess.QUEEN: [chess.D1, chess.D8],
                       chess.KING: [chess.E1, chess.E8]}

    # Check if a square is one of the initial squares for the piece type
    def is_initial_square(square, piece):
        if square in INITIAL_SQUARES[piece.piece_type]:
            return True
        else:
            return False

    # Checking if the pieces are moved from their initial position
    for move in board.move_stack:
        piece = board.piece_at(move.from_square)
        if piece is not None and is_initial_square(move.from_square, piece):
            points -= 0.5

    # Checking if the enemy is checked
    if board.is_check():
        points += 1

    # Checks checkmates
    if board.is_checkmate():
        if board.turn == chess.WHITE:
            points -= 1000
        else:
            points += 1000

    # Evaluate possible stalemates
    if board.is_stalemate():
        points += 0.5

    return points

In [4]:
def print_board(board):
    piece_map = board.piece_map()
    for rank in range(7, -1, -1):
        for file in range(8):
            square = chess.square(file, rank)
            piece = piece_map.get(square)
            if piece:
                piece_name = White[piece.piece_type]
                if piece.color == chess.BLACK:
                    piece_name = Black[piece.piece_type]
                print(piece_name, end=" ")
            else:
                print("--", end=" ")
        print()
    print()

In [5]:
def minimax(board, depth, alpha, beta, maxi):

    # at depth 0 runs the analyze state function to get the numeric value for the state
    if depth == 0 or board.is_game_over():
        return analyzeState(board)
    # checks the maximizing player
    if maxi:
        max_result = float('-inf')
        for step in board.legal_moves:
            board.push(step)
            result = minimax(board, depth - 1, alpha, beta, False)
            board.pop()
            max_result = max(max_result, result)
            alpha = max(alpha, result)
            if beta <= alpha:
                break
        return max_result
    # checks the minimizing player
    else:
        min_result = float('inf')
        for step in board.legal_moves:
            board.push(step)
            result = minimax(board, depth - 1, alpha, beta, True)
            board.pop()
            min_result = min(min_result, result)
            beta = min(beta, result)
            if beta <= alpha:
                break
        return min_result

In [6]:
# set up function for the min max function that initializes all required values and retuens the answer
def getBestMove(board, depth):
    bestMove = None
    maxResult = float('-inf')
    alpha = float('-inf')
    beta = float('inf')
    for move in board.legal_moves:
        board.push(move)
        result = minimax(board, depth - 1, alpha, beta, False)
        board.pop()
        if result > maxResult:
            maxResult = result
            bestMove = move
        alpha = max(alpha, result)
    return bestMove

def print_board(board):
    piece_map = board.piece_map()
    for rank in range(7, -1, -1):
        for file in range(8):
            square = chess.square(file, rank)
            piece = piece_map.get(square)
            if piece:
                piece_name = White[piece.piece_type]
                if piece.color == chess.BLACK:
                    piece_name = Black[piece.piece_type]
                print(piece_name, end=" ")
            else:
                print("--", end=" ")
        print()
    print()

In [7]:
# takes notes all of the moves
allmoves = []


while not chessboard.is_game_over():
    print()
    print_board(chessboard)
    print(chessboard.legal_moves)
    try:
        x = input('Enter your move : ')
        move = chessboard.parse_san(x)
        if move in chessboard.legal_moves:
            chessboard.push(move)
            allmoves.append(move)
            print('Score:', analyzeState(chessboard))
        else:
            print('Invalid move: not a legal move.')
            continue
    except ValueError as e:
        print(f'Invalid input: {e}')
        continue

    computer_move = getBestMove(chessboard, 3)
    allmoves.append(computer_move)
    chessboard.push(computer_move)
    print('Computer move:', computer_move, ', Score:', analyzeState(chessboard))

print('Game over.')
print(allmoves)


♜ ♞ ♝ ♛ ♚ ♝ ♞ ♜ 
♟ ♟ ♟ ♟ ♟ ♟ ♟ ♟ 
-- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- 
♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙ 
♖ ♘ ♗ ♕ ♔ ♗ ♘ ♖ 

<LegalMoveGenerator at 0x25b311b5a50 (Nh3, Nf3, Nc3, Na3, h3, g3, f3, e3, d3, c3, b3, a3, h4, g4, f4, e4, d4, c4, b4, a4)>
Invalid input: invalid san: ''

♜ ♞ ♝ ♛ ♚ ♝ ♞ ♜ 
♟ ♟ ♟ ♟ ♟ ♟ ♟ ♟ 
-- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- 
♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙ 
♖ ♘ ♗ ♕ ♔ ♗ ♘ ♖ 

<LegalMoveGenerator at 0x25b311ae650 (Nh3, Nf3, Nc3, Na3, h3, g3, f3, e3, d3, c3, b3, a3, h4, g4, f4, e4, d4, c4, b4, a4)>
Score: 0
Computer move: e7e6 , Score: 0

♜ ♞ ♝ ♛ ♚ ♝ ♞ ♜ 
♟ ♟ ♟ ♟ -- ♟ ♟ ♟ 
-- -- -- -- ♟ -- -- -- 
-- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- -- 
-- -- -- -- -- -- -- ♘ 
♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙ 
♖ ♘ ♗ ♕ ♔ ♗ -- ♖ 

<LegalMoveGenerator at 0x25b3134be50 (Ng5, Nf4, Ng1, Rg1, Nc3, Na3, g3, f3, e3, d3, c3, b3, a3, g4, f4, e4, d4, c4, b4, a4)>
Invalid input: illegal san: 'h3' in rnbqkbnr/ppp